In [ ]:
!pip install -qU datasets accelerate
!pip install -qU transformers[torch]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd
df = pd.read_csv("./Filtered_labelled_state_transcripts (1).csv")
encoder = preprocessing.LabelEncoder()
df.rename(columns = {'Label': 'labels'}, inplace=True)
#print(df.head())
encoder.fit(df['labels'])
df['labels']=encoder.transform(df['labels'])
print(df.head())
X_train, X_test, y_train, y_test = train_test_split(df['transcripts'], df['labels'], test_size=0.2, random_state=20)
train = pd.DataFrame([X_train, y_train]).T
train.set_index('transcripts').dropna().to_csv('train.csv')

test = pd.DataFrame([X_test, y_test]).T
test.set_index('transcripts').dropna().to_csv('test.csv')


   Unnamed: 0      videoId                                              title  \
0           1  Gc1tf5U7BuU  BJP Manifesto Promises To Secure &amp; Protect...   
1           2  1BAgeFCwR3o  Reporters Project: Andhra Pradesh’s Politics O...   
2           3  HKCBxHhnfC4  Reporters Project: Giving Voice To The Real Is...   
3           4  iLlUz3a4zDg  “I Definitely Don&#39;t Fight A War To Lose” –...   
4           5  sLdneSyrx7c  NaMo TV launched, Rahul Gandhi To Contest From...   

                                         description          published_at  \
0  BattleOf2019 #ElectionsWithNews18 #LokSabhaEle...  2019-04-08T14:19:22Z   
1  BattleOf2019 #ElectionsWithNews18 #LokSabhaEle...  2019-04-07T09:22:24Z   
2  BattleOf2019 #ElectionsWithNews18 #LokSabhaEle...  2019-04-07T09:20:19Z   
3  On Off Centre Anuradha SenGupta is talking to ...  2019-04-06T10:28:52Z   
4  NaMo TV channel launched, Rahul Gandhi to cont...  2019-04-06T05:00:02Z   

      channel                               

In [ ]:
from datasets import load_dataset, load_metric
import numpy as np
model_metric = load_metric('accuracy')

<ipython-input-3-0ac10517b774>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  model_metric = load_metric('accuracy')


In [ ]:
def compute_metric(model_pred):
  print(model_pred)
  preds, labels = model_pred
  # else: preds, labels, acc = model_pred
  preds = np.argmax(preds, -1)
  return model_metric.compute(predictions=preds, references=labels)

In [ ]:
dataset = load_dataset("csv", data_files="train.csv")
test_set = load_dataset("csv", data_files="test.csv")
print(dataset['train']['transcripts'][10])
set(dataset['train']['labels']), dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

good evening call roadblock programming well basically step away news hour prioritize something else tonight we're gonna prioritize mental health we're going talk quite cheeky smuggle conversations primetime programming special show show we're going weekends we're going 9 p.m. primetime among important issues Indians facing today family members colleagues friends spouses need know much mental health keep loved ones safe far many times someone seems happy seems successful seems like everything commit suicide pick tell-tale signs maybe paying close enough attention maybe know look w-h-o statistics say 5 crore Indians suffer mental health issues likely number lot higher many families deal problems embarrassed talk children school today diagnosed problems like ADHD learning disabilities parents embarrassed acknowledge child may problem delayed diagnosis means delayed treatment perhaps lifetime suffering phone lines open today anyone wants pick phone ask us question panel guests studio amaz

({0, 1, 2},
 DatasetDict({
     train: Dataset({
         features: ['transcripts', 'labels'],
         num_rows: 937
     })
 }))

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels = 3,ignore_mismatched_sizes=True)

def tokenize_fn(samples):
  return tokenizer(samples['transcripts'], max_length=512, truncation=True, padding='max_length')

tokenized_train_set = dataset['train'].map(tokenize_fn, batched=True)
tokenized_test_set = test_set['train'].map(tokenize_fn, batched=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/937 [00:00<?, ? examples/s]

Map:   0%|          | 0/235 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    push_to_hub=False,
    label_names=['class'],
    fp16=True,
    save_strategy='no',
    do_eval=True,
    # eval_steps=10,
    logging_strategy="epoch"
)
# fp16 => For using GPU

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        loss = outputs.loss
        # print('----------------------------------', model(**inputs), '---------------------------------')
        return (loss, outputs) if return_outputs else loss


trainer = Trainer(model=model,
                  args=args,
                  train_dataset=tokenized_train_set,
                  eval_dataset=tokenized_test_set,
                  compute_metrics=compute_metric
                  )

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.831500,No log
2,0.669900,No log
3,0.595000,No log
4,0.498700,No log
5,0.408800,No log
6,0.344700,No log
7,0.267700,No log
8,0.227900,No log
9,0.178000,No log
10,0.112500,No log


TrainOutput(global_step=2360, training_loss=0.22462739237284257, metrics={'train_runtime': 404.2625, 'train_samples_per_second': 46.356, 'train_steps_per_second': 5.838, 'total_flos': 2482483321589760.0, 'train_loss': 0.22462739237284257, 'epoch': 20.0})

In [ ]:
pred = trainer.predict(tokenized_train_set)
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)
accuracy_score(a, y_train)

<ipython-input-9-7743a01b91be>:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)


0.9967982924226254

In [ ]:
pred = trainer.predict(tokenized_test_set)
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)
accuracy_score(a, y_test)

<ipython-input-10-28fc1102d561>:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)


0.676595744680851

In [ ]:
trainer.save_model('Saved_model_ROBERTA')

In [ ]:
!zip -r /content/Saved_model_ROBERTA.zip /content/Saved_model_ROBERTA

  adding: content/Saved_model_ROBERTA/ (stored 0%)
  adding: content/Saved_model_ROBERTA/training_args.bin (deflated 51%)
  adding: content/Saved_model_ROBERTA/config.json (deflated 51%)
  adding: content/Saved_model_ROBERTA/model.safetensors (deflated 7%)


In [ ]:
!mv '/content/Saved_model_ROBERTA.zip' '/content/drive/MyDrive'